In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
file_path = '../data/Coffee Shop Sales.xlsx'
df = pd.read_excel(file_path)

In [3]:
df

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
0,1,2023-01-01,07:06:11,2,5,Lower Manhattan,32,3.00,Coffee,Gourmet brewed coffee,Ethiopia Rg
1,2,2023-01-01,07:08:56,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
2,3,2023-01-01,07:14:04,2,5,Lower Manhattan,59,4.50,Drinking Chocolate,Hot chocolate,Dark chocolate Lg
3,4,2023-01-01,07:20:24,1,5,Lower Manhattan,22,2.00,Coffee,Drip coffee,Our Old Time Diner Blend Sm
4,5,2023-01-01,07:22:41,2,5,Lower Manhattan,57,3.10,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg
...,...,...,...,...,...,...,...,...,...,...,...
149111,149452,2023-06-30,20:18:41,2,8,Hell's Kitchen,44,2.50,Tea,Brewed herbal tea,Peppermint Rg
149112,149453,2023-06-30,20:25:10,2,8,Hell's Kitchen,49,3.00,Tea,Brewed Black tea,English Breakfast Lg
149113,149454,2023-06-30,20:31:34,1,8,Hell's Kitchen,45,3.00,Tea,Brewed herbal tea,Peppermint Lg
149114,149455,2023-06-30,20:57:19,1,8,Hell's Kitchen,40,3.75,Coffee,Barista Espresso,Cappuccino


In [4]:
print("Data inicial: ", min(df['transaction_date']))
print("Data final: ", max(df['transaction_date']))

Data inicial:  2023-01-01 00:00:00
Data final:  2023-06-30 00:00:00


In [5]:
# Verifica se tem valores nulos
## não tem
df.isnull().sum()

transaction_id      0
transaction_date    0
transaction_time    0
transaction_qty     0
store_id            0
store_location      0
product_id          0
unit_price          0
product_category    0
product_type        0
product_detail      0
dtype: int64

In [6]:
# Verifica o intervalo de tempo do dataset
## 6 meses completos
df["transaction_date"].describe()

count                           149116
mean     2023-04-15 11:50:32.173609984
min                2023-01-01 00:00:00
25%                2023-03-06 00:00:00
50%                2023-04-24 00:00:00
75%                2023-05-30 00:00:00
max                2023-06-30 00:00:00
Name: transaction_date, dtype: object

In [7]:
df.dtypes

transaction_id               int64
transaction_date    datetime64[ns]
transaction_time            object
transaction_qty              int64
store_id                     int64
store_location              object
product_id                   int64
unit_price                 float64
product_category            object
product_type                object
product_detail              object
dtype: object

In [8]:
# Verifica se na mesma transaction_id teria outros produtos associados
## não tem
df["transaction_id"].value_counts()

transaction_id
1         1
99641     1
99661     1
99662     1
99663     1
         ..
49812     1
49813     1
49814     1
49815     1
149456    1
Name: count, Length: 149116, dtype: int64

In [9]:
# Verifica se existe mais de uma loja em uma mesma localização
## não tem
df[["transaction_id", "store_location", "store_id"]].groupby([ "store_location", "store_id"]).count()

,,transaction_id
store_location,store_id,
Astoria,3,50599
Hell's Kitchen,8,50735
Lower Manhattan,5,47782


In [10]:
# Verifica se um product_id é vendido em mais de uma localização de loja
## sim

# Agrupa por product_id e store_location para verificar a presença de produtos em diferentes lojas
product_store_group = df.groupby(['product_id', 'store_location']).size().reset_index(name='counts')

# Conta o número de lojas em que cada product_id é vendido
product_store_count = product_store_group.groupby('product_id')['store_location'].count().reset_index(name='num_stores')

print("Total de product_id", len(df['product_id'].unique()))
print("Nro de product_id vendidos em apenas uma localização de loja: ", len(product_store_count[product_store_count['num_stores'] == 1]))
print("Nro de product_id vendidos em 2 localizações de loja: ", len(product_store_count[product_store_count['num_stores'] == 2]))
print("Nro de product_id vendidos nas 3 localizações de loja: ", len(product_store_count[product_store_count['num_stores'] == 3]))

Total de product_id 80
Nro de product_id vendidos em apenas uma localização de loja:  0
Nro de product_id vendidos em 2 localizações de loja:  1
Nro de product_id vendidos nas 3 localizações de loja:  79


In [11]:
# Verifica a quantidade de categorias distintas e as mais consumidas
df["product_category"].value_counts()

product_category
Coffee                58416
Tea                   45449
Bakery                22796
Drinking Chocolate    11468
Flavours               6790
Coffee beans           1753
Loose Tea              1210
Branded                 747
Packaged Chocolate      487
Name: count, dtype: int64

In [12]:
# Verifica a quantidade de tipos de produtos distintos e os mais consumidas
df["product_type"].value_counts()

product_type
Brewed Chai tea          17183
Gourmet brewed coffee    16912
Barista Espresso         16403
Hot chocolate            11468
Brewed Black tea         11350
Brewed herbal tea        11245
Scone                    10173
Organic brewed coffee     8489
Drip coffee               8477
Premium brewed coffee     8135
Pastry                    6912
Biscotti                  5711
Brewed Green tea          5671
Regular syrup             4979
Sugar free syrup          1811
Housewares                 526
Chai tea                   443
Organic Beans              415
Gourmet Beans              366
Premium Beans              336
Espresso Beans             319
Herbal tea                 305
Black tea                  303
Drinking Chocolate         266
Organic Chocolate          221
Clothing                   221
House blend Beans          183
Green tea                  159
Green beans                134
Name: count, dtype: int64

In [13]:
# Verifica se um product_id pode ter mais de um unit_price
## pode sim

product_price_variation = df.groupby('product_id')['unit_price'].nunique().reset_index()
product_price_variation = product_price_variation.rename(columns={'unit_price': 'unique_prices_count'})

# Filtra os product_ids que têm mais de um preço único
multiple_prices_products = product_price_variation[product_price_variation['unique_prices_count'] > 1]

# Exibe os resultados
print("Produtos com mais de um preço único:")
multiple_prices_products

Produtos com mais de um preço único:


,product_id,unique_prices_count
8,9,4
64,69,2
65,70,2
66,71,2
67,72,3
68,73,2
69,74,2
70,75,2
71,76,2
73,78,2


In [14]:
df[df['product_id'] == 9]

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail
3296,3326,2023-01-07,07:45:15,1,5,Lower Manhattan,9,28.0,Coffee beans,Organic Beans,Organic Decaf Blend
3302,3332,2023-01-07,07:50:42,1,5,Lower Manhattan,9,28.0,Coffee beans,Organic Beans,Organic Decaf Blend
3408,3438,2023-01-07,09:05:19,1,5,Lower Manhattan,9,28.0,Coffee beans,Organic Beans,Organic Decaf Blend
3448,3478,2023-01-07,09:42:52,1,5,Lower Manhattan,9,12.0,Coffee beans,Organic Beans,Organic Decaf Blend
3571,3601,2023-01-07,10:56:05,1,5,Lower Manhattan,9,12.0,Coffee beans,Organic Beans,Organic Decaf Blend
...,...,...,...,...,...,...,...,...,...,...,...
141305,141642,2023-06-24,08:17:19,1,5,Lower Manhattan,9,22.5,Coffee beans,Organic Beans,Organic Decaf Blend
142220,142557,2023-06-24,18:28:53,1,8,Hell's Kitchen,9,22.5,Coffee beans,Organic Beans,Organic Decaf Blend
142971,143308,2023-06-25,12:25:24,1,8,Hell's Kitchen,9,22.5,Coffee beans,Organic Beans,Organic Decaf Blend
143509,143846,2023-06-26,06:49:02,1,5,Lower Manhattan,9,22.5,Coffee beans,Organic Beans,Organic Decaf Blend
